In [28]:
from flask import Flask
from flask import request
from flask import jsonify
from flask_cors import CORS
import pandas as pd
import random
from sklearn.cluster import KMeans
from datetime import datetime, timedelta


In [29]:
app = Flask(__name__)  # Corrected __name__
CORS(app)

In [36]:
# Load dataset
df = pd.read_csv("C:/Users/Asus/Downloads/AI_Learning_Path_Dataset.csv")

In [37]:
# Preprocess
df.fillna({"BadgesEarned": 0, "ProgressTracking": df["ProgressTracking"].median()}, inplace=True)

In [38]:
# Clustering Model
kmeans = KMeans(n_clusters=3, random_state=42)
df["UserCluster"] = kmeans.fit_predict(df[["StudyHours", "ProgressTracking"]])

In [39]:
# Recommend Learning Paths
def recommend_learning_path(user_id):
    user_cluster = df[df["UserID"] == user_id]["UserCluster"].values[0]
    recommendations = df[df["UserCluster"] == user_cluster].sample(3)[["RecommendedCourse", "CareerGoal"]]
    return recommendations.to_dict(orient="records")

In [40]:
# Gamification Elements (Leaderboard, Badges, Streaks)
leaderboard = df.sort_values(by="ProgressTracking", ascending=False)[["UserID", "BadgesEarned", "ProgressTracking"]].head(10)

In [41]:
@app.route("/recommend", methods=["POST"])
def recommend():
    user_id = request.json["UserID"]
    return jsonify(recommend_learning_path(user_id))

In [42]:

@app.route("/leaderboard", methods=["GET"])
def get_leaderboard():
    return jsonify(leaderboard.to_dict(orient="records"))

In [44]:
if __name__ == "_main_":
    app.run(debug=True)